In [46]:
# coding: utf-8
import pickle
from datetime import datetime

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import sklearn
from sklearn.linear_model import LogisticRegression
from sklearn.metrics import accuracy_score

# Reading and Processing Propublica Data

In [47]:
#https://github.com/propublica/compas-analysis/blob/master/compas-scores-two-years.csv
df = pd.read_csv('recidivism-2-years.csv', index_col=0)

###### Recommended cleaning steps

In [48]:
# See cell 2 here https://github.com/propublica/compas-analysis/blob/master/Compas%20Analysis.ipynb
df = df[(df["is_recid"] != -1) & (df["days_b_screening_arrest"] <= 30) & (df["days_b_screening_arrest"] >= -30) & (df["c_charge_degree"] != "O") & (df["score_text"] != "N/A")]

In [49]:
did_recidivate = df['two_year_recid'].as_matrix()
dec_score = df['decile_score'].as_matrix()
racial_desc = df["race"]

# Retaining only those attributes allowed to be used in recidivism predictions and sentencing
columns_to_keep = ["sex", "age", "age_cat", "juv_fel_count", "juv_misd_count",
                   "juv_other_count", "priors_count", "c_charge_degree"]
df = df[columns_to_keep]

In [50]:
# Get categorical features and dump redundant
df_bin = pd.get_dummies(df)
df_bin = df_bin.drop('sex_Female', axis=1)
df_bin = df_bin.drop('c_charge_degree_M', axis=1)

X = df_bin.as_matrix()

In [51]:
# All equivalent in this binary case
def uncertainty_sample_proba(clf, x):
    probs = clf.predict_proba(x)
    return np.amax(probs, axis=1)

def top_2_dif(x):
    x = sorted(list(x))
    return x[-1] - x[-2]

def smallest_margin_proba(clf,x):
    probs = clf.predict_proba(x)
    probs = np.array(map(top_2_dif, list(probs)))
    return probs # x_pool[np.argsort(probs)[0:n]]
    
def label_entropy_proba(clf, x):
    probs = clf.predict_proba(x)
    probs = np.array(map(lambda row: sum(map(lambda y: -y*np.log2(y), row))))
    return probs

In [52]:
# Pool data
count = 0
from random import shuffle
idx = range(len(X))
shuffle(idx)
init_pool = idx[0:400]
valida_pool = idx[-2000:]
idx = idx[0:-2000]
outside_pool = set(idx).difference(init_pool)

In [53]:
def di_calc(clf, X, protected_attribute_indices, uncertainty_cutoff=None):
    """Calculate disparate impact given all minority class indices"""
    probs = uncertainty_sample_proba(clf, X)
    if uncertainty_cutoff is None:
        uncertainty_cutoff = np.median(probs)
    protected_majority = set(range(X.shape[0])) - set(protected_attribute_indices)
    protected_majority = list(protected_majority)
    certain = probs >= uncertainty_cutoff
    
    # As defined here: https://arxiv.org/pdf/1412.3756.pdf - check page 5.
    a = float(np.count_nonzero(certain[protected_attribute_indices] == False))
    c = float(np.count_nonzero(certain[protected_attribute_indices]))
    b = float(np.count_nonzero(certain[protected_majority] == False))
    d = float(np.count_nonzero(certain[protected_majority]))
    
    if a == 0:
        a = 0.00001
    if b == 0:
        b = 0.00001
    if c == 0:
        c = 0.00001
    if d == 0:
        d = 0.00001
    
    pos_likelihood_ratio = (d / (b + d)) / (c / (c + a))

    return 1 - 1/pos_likelihood_ratio

In [54]:
# Note protected attributes
race_Hispanic_ids = [index for index, is_class in enumerate((racial_desc == 'Hispanic').as_matrix()) if is_class]
race_Asian_ids =  [index for index, is_class in enumerate((racial_desc == 'Asian').as_matrix()) if is_class]
race_African_American =  [index for index, is_class in enumerate((racial_desc == 'African-American').as_matrix()) if is_class]
race_Caucasian = [index for index, is_class in enumerate((racial_desc == 'Caucasian').as_matrix()) if is_class]

male_ids = [index for index, is_class in enumerate((df['sex'] == 'Male').as_matrix()) if is_class]
female_ids = [index for index, is_class in enumerate((df['sex'] == 'Female').as_matrix()) if is_class]

In [ ]:
# Training run
startTime = datetime.now()

performances = []
perf_ns = []

clf = LogisticRegression(solver='sag', max_iter=1000, random_state=42,
                             multi_class='multinomial')
clf.fit(X[init_pool, :], did_recidivate[init_pool])
predictions = clf.predict(X[valida_pool, :])
performances.append(accuracy_score(predictions, did_recidivate[valida_pool]))
perf_ns.append(len(init_pool))

di_black = []
di_hispanic = []
di_asian = []
di_white = []

di_male = []
di_female = []

mean_certainties_black = []
mean_certainties_hispanic = []
mean_certainties_asian = []
mean_certainties_white = []

for i in range(2000):
    probs = uncertainty_sample_proba(clf, X[sorted(outside_pool), :])
    index = sorted(outside_pool)[np.argmin(probs)]
    init_pool.append(index)
    outside_pool.remove(index)
        
    clf.fit(X[init_pool, :], did_recidivate[init_pool])
    predictions = clf.predict(X[valida_pool, :])
    performances.append(accuracy_score(predictions, did_recidivate[valida_pool]))
    perf_ns.append(len(init_pool))
    
    # Record disparate impact of uncertainty
    di_black.append(di_calc(clf, X, race_African_American))
    di_hispanic.append(di_calc(clf, X, race_Hispanic_ids))
    di_asian.append(di_calc(clf, X, race_Asian_ids))
    di_white.append(di_calc(clf, X, race_Caucasian))
    
    di_male.append(di_calc(clf, X, female_ids))
    di_male.append(di_calc(clf, X, male_ids))
    
    # Record mean certainties
    probs = uncertainty_sample_proba(clf, X)
    mean_certainties_black.append(np.mean(probs[race_African_American]))
    mean_certainties_hispanic.append(np.mean(probs[race_Hispanic_ids]))
    mean_certainties_asian.append(np.mean(probs[race_Asian_ids]))
    mean_certainties_white.append(np.mean(probs[race_Caucasian]))

    assert(len(performances)==len(perf_ns))

print datetime.now() - startTime

In [ ]:
run_num = 0
with open('all_performances_{}.pl'.format(run_num), 'wb') as f:
    pickle.dump([di_black,di_hispanic, di_asian, di_white, di_male, di_female, mean_certainties_black, mean_certainties_hispanic, mean_certainties_asian, mean_certainties_white, performances], f)

In [ ]:
plt.figure(figsize=(16,9), dpi=300)
plt.plot(perf_ns[:-1], mean_certainties_black)
plt.plot(perf_ns[:-1], mean_certainties_hispanic)
plt.plot(perf_ns[:-1], mean_certainties_asian)
plt.plot(perf_ns[:-1], mean_certainties_white)
plt.xlabel('Labeled Pool Size')
plt.ylabel('Mean Certainty')
plt.legend(['Black', 'Hispanic', 'Asian', 'White'], loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(16,9), dpi=300)
plt.plot(perf_ns[:-1], mean_certainties_black)
plt.plot(perf_ns[:-1], mean_certainties_hispanic)
plt.plot(perf_ns[:-1], mean_certainties_asian)
plt.plot(perf_ns[:-1], mean_certainties_white)
plt.xlabel('Labeled Pool Size')
plt.ylabel('Mean Certainty')
plt.legend(['Black', 'Hispanic', 'Asian', 'White'], loc='best')
plt.show()

In [ ]:
plt.figure(figsize=(16, 9), dpi=300)
plt.plot(perf_ns[1:], di_black)
plt.plot(perf_ns[1:], di_hispanic)
plt.plot(perf_ns[1:], di_asian)
plt.plot(perf_ns[1:], di_white)
plt.legend(['Black', 'Hispanic', 'Asian', 'White'], loc='best')
plt.show()

In [ ]:


# In[108]:

plt.figure(figsize=(16,9), dpi=200)
plt.plot(perf_ns[1:], di_black)
plt.plot(perf_ns[1:], di_hispanic)
plt.plot(perf_ns[1:], di_white)
plt.legend(['Black', 'Hispanic', 'White'], loc='best')
# plt.savefig('LongerTrialUncertaintyBias_WIDE2.eps', format='eps')
plt.show()

In [ ]:
plt.figure(figsize=(8,8), dpi=200)
plt.plot(perf_ns[1:], di_black, color=plt.cm.Paired(0.1))
plt.plot(perf_ns[1:], di_hispanic, color=plt.cm.Paired(0.2))
plt.plot(perf_ns[1:], di_white, color=plt.cm.Paired(0.4))
# plt.legend(['Black', 'Hispanic', 'White'], loc='best')
plt.xlabel('Labeled Pool Size')
plt.ylabel('Uncertainty Bias')
# plt.savefig('FINAL_GRAPHX/Black_Hispanic_White_Plain_AL_Comparison_Final_FULL_WIDE.eps', format='eps')
# plt.legend(['UNCERTAINTY BIAS'])
plt.show()

In [ ]:
plt.figure(figsize=(8,8), dpi=200)
fig, ax1 = plt.subplots(figsize=(8,8), dpi=200)
plt.plot(perf_ns[:-1], di_black, color=plt.cm.Paired(0.1))
plt.plot(perf_ns[:-1], di_hispanic, color=plt.cm.Paired(0.2))
plt.plot(perf_ns[:-1], di_white, color=plt.cm.Paired(0.4))
plt.plot(perf_ns, performances, color=(0, 0, 0))
# plt.legend(['Black', 'Hispanic', 'White'], loc='best')
plt.xlabel('Labeled Pool Size')
plt.ylabel('Uncertainty Bias')
plt.show()

In [ ]:
choice_counts = [i[0] for i in choice_counts]


# In[ ]:

vectors = []
for choice in choice_counts:
    vectors.append([0 if index != choice else 1 for index in range(len(set(choice_counts)))])


# In[ ]:

plt.cm.Paired(i)


# In[ ]:

plt.figure(figsize=(20,10))

for i in range(np.array(vectors).shape[1]):
    plt.plot(perf_ns, np.cumsum(vectors, axis=0)[:, i], color=plt.cm.Paired(float(i)/clust_number))


# In[ ]:

plt.cm.Paired(1.0)


# In[ ]:

plt.show()


# In[ ]:

# center_labels = []
# for center in cluster.cluster_centers_:
#     center_labels.append(np.argsort(np.round(np.absolute(center), decimals=2))[-2:])


# In[ ]:

init_pool


# In[ ]:

race_cumulative_sums[400:, i].shape


# In[ ]:

plt.figure(figsize=(8,8), dpi=400)

race_cumulative_sums = []
for i in init_pool:
    if i in race_Hispanic_ids:
        race_cumulative_sums.append(np.array([1,0,0,0]))
    elif i in race_Asian_ids:
        race_cumulative_sums.append(np.array([0,1,0,0]))
    elif i in race_African_American:
        race_cumulative_sums.append(np.array([0,0,1,0]))
    elif i in race_Caucasian:
        race_cumulative_sums.append(np.array([0,0,0,1]))
    else:
        race_cumulative_sums.append(np.array([0,0,0,0]))
    
race_cumulative_sums = np.cumsum(race_cumulative_sums, axis=0)

for i in range(4):
     plt.plot(perf_ns[:-1], race_cumulative_sums[400:, i], color=plt.cm.Paired(float(i)/4), linewidth=3)

plt.xlabel("Total training set size")
plt.ylabel("Number of samples queried from pool")
plt.legend(['Hispanic', 'Asian', 'Black', 'White'], loc='best')
plt.savefig('Race_query_countsBT.eps', format='eps')


# In[ ]:

import matplotlib

font = {'family' : 'normal',
        'weight' : 'normal',
        'size'   : 16}

matplotlib.rc('font', **font)


# In[ ]:

plt.figure(figsize=(8,8), dpi=400)

race_cumulative_sums = []
for i in init_pool:
    if i in race_Hispanic_ids:
        race_cumulative_sums.append(np.array([1,0,0,0]))
    elif i in race_African_American:
        race_cumulative_sums.append(np.array([0,0,1,0]))
    elif i in race_Caucasian:
        race_cumulative_sums.append(np.array([0,0,0,1]))
    else:
        race_cumulative_sums.append(np.array([0,0,0,0]))
    
race_cumulative_sums = np.cumsum(race_cumulative_sums, axis=0)

colors = [0.2, 0.1, 0.4]
for i, total_head_count, color in zip([0, 2, 3], [637, 3696, 2454], colors):
     plt.plot(perf_ns[:-1], race_cumulative_sums[400:, i]/float(total_head_count), color=plt.cm.Paired(color), linewidth=2.5)

plt.xlabel("Total training set size")
plt.ylabel("Proportion of total population in dataset")
plt.legend(['Hispanic', 'Black', 'White'], loc='best')
plt.savefig('Race_query_proportionBT.eps', format='eps')


# In[ ]:

population_counts = [637,32,3696,2454]


# In[ ]:

plt.plot(race_cumulative)


# In[ ]:

race_Hispanic_ids = [index for index, is_class in enumerate((df['race'] == 'Hispanic').as_matrix()) if is_class]
race_Asian_ids =  [index for index, is_class in enumerate((df['race'] == 'Asian').as_matrix()) if is_class]
race_African_American =  [index for index, is_class in enumerate((df['race'] == 'African-American').as_matrix()) if is_class]
race_Caucasian = [index for index, is_class in enumerate((df['race'] == 'Caucasian').as_matrix()) if is_class]


# In[ ]:

# legend = [list(df_exp.columns)[i] + '        &         ' + list(df_exp.columns)[v] for i,v  in center_labels]
legend = []

for i in center_labels:
    label = ''
    
    for v in i:
        label += list(df_exp.columns)[v]
        if v != i[-1]:
            label += '   &     '
        else:
            pass
    if label == '':
        label = 'Many sources'
    legend.append(label)


# In[ ]:

legend


# In[ ]:

plt.figure(figsize=(12,12), dpi=5)


for i in range(np.array(vectors).shape[1]):
    plt.plot(perf_ns, np.cumsum(vectors, axis=0)[:, i], color=plt.cm.Paired(float(i)/clust_number))

# plt.set_linewidth(44)
plt.xlabel("Total training set size")
plt.ylabel("Number of samples from a cluster",)
plt.legend(legend, loc='best')


# In[ ]:

plt.show()


# In[ ]:

# Highly explored curves
high_indices = []
high_legend = []

for index, (label, count) in enumerate(zip(legend, np.cumsum(vectors, axis=0)[-1])):
    if count >= 90:
        high_indices.append((index, count, label))
        high_legend.append(label)
        
high_indices, counts, high_legend = zip(*sorted(high_indices, key=lambda x:x[1], reverse=True))
        
        
        
plt.figure(figsize=(8,8))    
    
for i in high_indices:
        plt.plot(perf_ns, np.cumsum(vectors, axis=0)[:, i], color=plt.cm.Paired(float(i)/clust_number), linewidth=2.5)

# plt.set_linewidth(44)
plt.xlabel("Total training set size")
plt.ylabel("n")
plt.legend(high_legend, loc='best')
plt.savefig("HighFreqCertaintyCounts.eps", format='eps')


# In[ ]:

# Highly explored curves
# high_indices = []
# high_legend = []
# for index, (label, count) in enumerate(zip(legend, np.cumsum(vectors, axis=0)[-1])):
#     if count >= 100:
#         high_indices.append(index)
#         high_legend.append(label)
        
plt.figure(figsize=(8,8))    
    
for i in high_indices:
        plt.plot(perf_ns, np.array(cluster_dis)[:, i], color=plt.cm.Paired(float(i)/clust_number))

# plt.set_linewidth(44)
plt.xlabel("Total training set size")
plt.ylabel("Uncertainty Bias")
# plt.legend(high_legend, loc='best')

plt.savefig("HighFreqCertaintyBias_NO_LEGEND.eps", format='eps')


# In[ ]:


from matplotlib.font_manager import FontProperties

fontP = FontProperties()
fontP.set_size('medium')

# Highly underexplored curves
high_indices = []
high_legend = []
for index, (label, count) in enumerate(zip(legend, np.cumsum(vectors, axis=0)[-1])):
    if count <= 20:
        high_indices.append(index)
        high_legend.append(label)
        
plt.figure(figsize=(10, 20))    
for i in range(np.array(vectors).shape[1]):
    if i in high_indices:
        plt.plot(perf_ns, np.cumsum(vectors, axis=0)[:, i], color=plt.cm.Paired(float(i)/clust_number))

# plt.set_linewidth(44)
plt.xlabel("Total training set size")
plt.ylabel("n")
plt.legend(high_legend, loc='best', prop = fontP)
plt.savefig("LowFreqCertaintyCounts.eps", format='eps')


# In[ ]:

# Highly explored curves
high_indices = []
high_legend = []
for index, (label, count) in enumerate(zip(legend, np.cumsum(vectors, axis=0)[-1])):
    if count <= 20:
        high_indices.append(index)
        high_legend.append(label)
        
plt.figure(figsize=(8,8))    
    
for i in range(np.array(vectors).shape[1]):
    if i in high_indices:
        plt.plot(perf_ns, np.array(cluster_dis)[:, i], color=plt.cm.Paired(float(i)/clust_number))

# plt.set_linewidth(44)
plt.xlabel("Total training set size")
plt.ylabel("Uncertainty Bias")
plt.legend(high_legend, loc='best')

plt.savefig("LowFreqCertaintyBias_TALL.eps", format='eps')


# In[ ]:

# Highly explored curves
high_indices = []
high_legend = []
for index, (label, count) in enumerate(zip(legend, np.cumsum(vectors, axis=0)[-1])):
    if count <= 20:
        high_indices.append(index)
        high_legend.append(label)
        
plt.figure(figsize=(30,40))    
    
for i in range(np.array(vectors).shape[1]):
    if i in high_indices:
        plt.plot(perf_ns, np.cumsum(vectors, axis=0)[:, i], color=plt.cm.Paired(float(i)/clust_number))

# plt.set_linewidth(44)
plt.xlabel("Total training set size", fontproperties=font_prop)
plt.ylabel("n", fontproperties=font_prop)
plt.legend(high_legend, prop=font_prop, loc='upper left')


# In[ ]:

plt.show()


# In[ ]:




# In[ ]:

plt.plot(perf_ns, performances)
plt.show()


# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:




# In[ ]:





In [ ]:
di_calc(clf, X, female_ids)
